In [1]:
%%time
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 

from binance import AsyncClient, BinanceSocketManager, Client, ThreadedWebsocketManager
import Artha.configs.binance_config as c 
key = c.apis[0][0]
secret = c.apis[0][1]
client = Client(key, secret)

import asyncio
import time
import nest_asyncio
nest_asyncio.apply()

# client.get_historical_klines("ETHBTC", "1h", "1 Dec, 2017", "1 Jan, 2018")
import pandas as pd
pd.options.display.float_format = '{:.9f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# from Artha.mentions import *
from datetime import datetime, timedelta
import numpy as np

from Artha.trading import *

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot

CPU times: user 1.13 s, sys: 217 ms, total: 1.34 s
Wall time: 1.95 s


In [3]:
df = load_klines("ETHBTC")

In [7]:
%%time
ticker, time_frame = "ETHBTC", "4h"
# klines = get_klines(ticker, time_frame, "01/01/2020 00:00:00")
# df = get_klines_df(klines)
ichimoku(df)
# save_klines(df, "ETHBTC")

CPU times: user 151 ms, sys: 5.16 ms, total: 156 ms
Wall time: 156 ms


In [10]:
%%time

d = df 
fig = setup_ichi_graph(d, ticker, time_frame, cloud=True, tkc=True)
        
# fig.update_layout(
#     title='The Great Recession',
#     yaxis_title='AAPL Stock',
#     shapes = [dict(
#         x0='2016-12-09', x1='2016-12-09', y0=0, y1=1, xref='x', yref='paper',
#         line_width=2)],
#     annotations=[dict(
#         x='2016-12-09', y=0.05, xref='x', yref='paper',
#         showarrow=False, xanchor='left', text='Increase Period Begins')]
# )
iplot( fig, filename = 'candlestick-ichimoku' )

CPU times: user 1.17 s, sys: 17.5 ms, total: 1.18 s
Wall time: 1.2 s


In [2]:
markets = get_quote_dict()
btc_markets = [tick+asset for asset in list(markets.keys())[:1] for tick in markets[asset]]
concurrent_download_klines(btc_markets[1:15], "03/01/2021 00:00:00")

In [ ]:
async def single(name):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ts = bm.kline_socket(name, interval="5m")
    # then start receiving messages
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            print(res)
   
    await client.close_connection()

async def multiple (asset_names):
    client = await AsyncClient.create()
    bm = BinanceSocketManager(client, user_timeout = 20)
    # start any sockets here, i.e a trade socket
    ms = bm.multiplex_socket(asset_names)
    # then start receiving messages
    async with ms as tscm:
        while True:
            res = await tscm.recv()
            print(res["data"]["k"])
   
    await client.close_connection()

In [ ]:
import Artha.crypto_data as crypto
markets = crypto.get_market_dict()
print("BTC" in [markets.keys()])
print("BTC" in [i for i in crypto.get_invert_dict(markets).keys()])
s_type = "@kline_5m"
btc_markets = [(tick+asset+s_type).lower() for asset in list(markets.keys())[:1] for tick in markets[asset]]

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(multiple(btc_markets[:3]))
print("done")

In [ ]:
symbol = 'BNBBTC'

twm = ThreadedWebsocketManager(api_key=key, api_secret=secret)
# start is required to initialise its internal loop
twm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

twm.start_kline_socket(callback=handle_socket_message, symbol=symbol)

# multiple sockets can be started
twm.start_depth_socket(callback=handle_socket_message, symbol=symbol)

# or a multiplex socket can be started like this
# see Binance docs for stream names
streams = ['BNBBTC@miniTicker', 'BNBBTC@bookTicker']
twm.start_multiplex_socket(callback=handle_socket_message, streams=streams)

In [ ]:
# incorporated social media data
start_date, window_count = "03/01/2021 00:00:00", 63

dates = [from_datetime(to_datetime(start_date)+timedelta(days=i)) for i in range(window_count)]

pdrank = pd.read_csv("pgrankdata.csv")
pdrank['Date'] = pd.to_datetime(pdrank['Unnamed: 0'])
pdrank = pdrank.set_index("Date").drop(['Unnamed: 0'], axis = 1)

i = 0
ticker = "DOGE"

while i<len(dates)-1:
    if pdrank[ticker][i]!=0:
        di = pdrank[ticker][i+1]/pdrank[ticker][i]
        if di> 1.1:
            print(i, di, pdrank[ticker][i+1], pdrank[ticker][i])
            print(pdrank[ticker].index[i+1])
    else:
        if pdrank[ticker][i+1] > 0:
            print("new", i, pdrank[ticker][i+1])
    i+=1

In [ ]:
# direct from neo

inc = 32
wind = 30

if ticker == "BTC":
    asset = ticker+"USDT"
else:
    asset = ticker+"BTC"

interval = '2h'
newest, oldest = pdrank[ticker].index[inc-wind: inc+wind][-1].to_pydatetime().timestamp() * 1000, pdrank[ticker].index[inc-wind: inc+wind][0].to_pydatetime().timestamp() * 1000, 
df = crypto.get_klines_df(asset, interval, str(oldest), str(newest))
df.keys()


plt.rcParams['figure.figsize'] = [10, 5]
ts = df["Volume"].astype(float) #.pct_change()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()


ax1.plot(pdrank[ticker].index[inc-wind: inc+wind], pdrank[ticker][inc-wind: inc+wind], color='b')

ax2.plot(ts.index, ts, color='g')

ax1.set_xlabel('Time')
ax1.set_ylabel(ticker+" PageRank Score", color='b')
ax2.set_ylabel(ticker +" Price", color='g')
# plt.plot([inc]*2, range(-1,1))
plt.title(asset+" "+from_datetime(pdrank[ticker].index[inc].to_pydatetime())[:-9])
plt.gcf().autofmt_xdate()